In [1]:
## Employment rates in Baltimore NIIF areas - 2021

In [ ]:
# From https://www.census.gov/data/developers/data-sets/acs-5year.html
# ACS has non-overlapping datasets that allow comparisons of current ACS data to past ACS data.  
# The 2017-2021 ACS 5-Year estimates can be compared with 2012-2016 ACS 5-Year estimates. 

In [2]:
### Census Data (2021:ACS-5 year) 
# NIIF census tract crosswalk file provided by Patrick/ GIS Analyst


In [3]:
# From https://www.census.gov/programs-surveys/acs/guidance/comparing-acs-data.html:
# "Due to the impact of the COVID-19 pandemic, the Census Bureau changed the 2020 ACS release. 
# Instead of providing the standard 1-year data products, the Census Bureau released experimental estimates from the 1-year data. 
# Data users should not compare 2020 ACS 1-year experimental estimates with any other data.""

In [4]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census

# Census & gmaps API Keys

from config import (api_key)
c = Census(api_key, year=2021)

# Configure gmaps
#gmaps.configure(api_key=gkey)

In [5]:
import os

In [6]:
os.getcwd()

'C:\\Users\\Jenkir\\Desktop\\BDC\\Baltimore_Together\\measuring-success-metrics\\BT-equity-metric-analysis'

In [7]:
# need to change the working directory; be sure that the config file for the census API is also in this folder
os.chdir('C:\\Users\\Jenkir\\Desktop\\BDC\\Baltimore_Together\\measuring-success-metrics\\BT-equity-metric-analysis')


In [8]:
os.getcwd()

'C:\\Users\\Jenkir\\Desktop\\BDC\\Baltimore_Together\\measuring-success-metrics\\BT-equity-metric-analysis'

In [9]:
pd.set_option('display.max_rows', 200)

In [10]:
pd.set_option('max_colwidth', 100)

In [11]:
# ACS Table codes to use in cell below;  Universe: Population 16 years and over 
# B23025_002E	Estimate!!Total:!!In labor force:	B23025_002EA,B23025_002M,B23025_002MA
# B23025_003E	Estimate!!Total:!!In labor force:!!Civilian labor force
# B23025_004E	Estimate!!Total:!!In labor force:!!Civilian labor force:!!Employed	B23025_004EA,B23025_004M,B23025_004MA
# B23025_005E	Estimate!!Total:!!In labor force:!!Civilian labor force:!!Unemployed	B23025_005EA,B23025_005M,B23025_005MA


In [12]:
# Run Census Search to retrieve data on all tracts 
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels

census_data = c.acs5.state_county(("NAME", "B19013_001E", "B01003_001E",
                          "B02001_003E",
                          "B02001_002E",
                          "B19001_001E",               
                          "B17001_002E",
                          "B23025_001E",
                          "B23025_002E",
                          "B23025_004E",
                          "B23025_005E"), 
                          state_fips = '24',
                          county_fips = '510'
                          )                  
                                          
# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B02001_002E": "Pop. white",
                                      "B02001_003E": "Pop. Black",
                                      "B19001_001E": "Total households",
                                      "B23025_004E": "# employed, age 16 and over",
                                      "B19013_001E": "Median hh Income",
                                      "B23025_001E": "Population age 16 and over",
                                      "B23025_002E": "In labor force",
                                      "B23025_005E": "# unemployed age 16 and over",
                                      "B17001_002E": "Poverty Count",
                                      "NAME": "Name"})
census_pd.head()

UnsupportedYearException: Geography is not available in 2021. Available years include (2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009)

In [ ]:
census2 = census_pd.copy()
census2.head()

In [ ]:
# Add in Poverty Rate (Poverty Count / Population)
census2["Poverty Rate"] = 100 * \
    census2["Poverty Count"].astype(
        int) / census2["Population"].astype(int)
census2

In [ ]:
# Add in employment rate (# employed, age 16 and over / Population age 16 and over)
census2["Employment Rate"] = 100 * \
    census2["# employed, age 16 and over"].astype(
        int) / census2["Population age 16 and over"].astype(int)
census2

In [ ]:
# Create Geographic Identifier ("GEOID") for each census tract by adding state fips code + county fips code + census tract code
# see https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html#:~:text=The%20full%20GEOID%20for%20many,codes%2C%20in%20which%20they%20nest.
census2["GEOID"] = census2['state'] + census2['county'] + census2['tract']
census2

In [ ]:
# GIS Analyst, Patrick, provided a csv (created from ARcGIS) that contains a key to match Baltimore's NIIF areas with census tracts

# Store filepath in a variable
NIIFs = "./raw_data/NIIF_Tracts.csv"

# Read the file with the pandas library
NIIFs_df = pd.read_csv(NIIFs)
NIIFs_df.dtypes

In [ ]:
# Change data type of the "GEOID" column so that it's a string/ text and not an integer
NIIFs_df["GEOID"] = NIIFs_df["GEOID"].astype('str')
NIIFs_df.dtypes

In [ ]:
# merge the census2 dataframe with the NIIFs_df dataframe on the common column "GEOID"

NIFF_df = pd.merge(
    census2, NIIFs_df, on="GEOID")

# remove any columns with NaN ("Not a Number") - used for missing values, by using .dropna()
#NIFF_df = NIFF_df.dropna()
NIFF_df

In [ ]:
# Remove extraneous columns
NIFF_df = NIFF_df.drop(["Name","NAMELSAD", "state", "county"], axis=1)
NIFF_df

In [ ]:
NIFF_df.dtypes

In [ ]:
NIFF_employment = NIFF_df.copy()


In [ ]:
NIFF_employment = NIFF_employment[['Population age 16 and over', '# employed, age 16 and over']]
NIFF_employment

In [ ]:
#  sum the population age 16 and over and # employed , age 16 and over for all of the census tracts that are in NIIF boundary
# see https://sparkbyexamples.com/pandas/pandas-sum-dataframe-columns/#:~:text=To%20sum%20given%20or%20list,to%20get%20the%20total%20sum.
df = NIFF_employment.sum(axis=0)
df2 = pd.DataFrame(df)
df2

In [ ]:
NIIF_employment = df2.transpose()
NIIF_employment

In [ ]:

NIIF_employment["employment rate"] = 100 * \
    NIIF_employment["# employed, age 16 and over"].astype(
        int) / NIIF_employment["Population age 16 and over"].astype(
        int) 

NIIF_employment

In [ ]:
# Round rate to one decimal point
NIIF_employment["employment rate"] = NIIF_employment["employment rate"].astype(float).round(1)
NIIF_employment

In [ ]:
# Export file as a CSV, without the Pandas index, but with the header
# Do not run this last code block until you have all previous code blocks in their final form:

# NIIF_employment.to_csv("NIIF_employment.csv", index = False, header=True)